In [56]:
import pandas as pd
import geopandas as gpd
import movingpandas as mpd
import matplotlib.pyplot as plt

from geopandas import GeoDataFrame, read_file
from shapely.geometry import Point, LineString, Polygon
from datetime import datetime, timedelta
from holoviews import opts, dim

import warnings
warnings.filterwarnings('ignore')
mpd.show_versions()


MovingPandas 0.13.rc1

SYSTEM INFO
-----------
python     : 3.9.15 | packaged by conda-forge | (main, Nov 22 2022, 08:55:37)  [Clang 14.0.6 ]
executable : /opt/anaconda3/envs/skmob/bin/python
machine    : macOS-13.0.1-x86_64-arm-64bit

GEOS, GDAL, PROJ INFO
---------------------
GEOS       : None
GEOS lib   : None
GDAL       : 3.6.1
GDAL data dir: /opt/anaconda3/envs/skmob/share/gdal
PROJ       : 9.1.0
PROJ data dir: /opt/anaconda3/share/proj

PYTHON DEPENDENCIES
-------------------
geopandas  : 0.12.2
pandas     : 1.5.2
fiona      : 1.8.22
numpy      : 1.24.0
shapely    : 2.0.0
rtree      : 1.0.1
pyproj     : 3.4.1
matplotlib : 3.6.2
mapclassify: 2.4.3
geopy      : 2.3.0
holoviews  : 1.15.3
hvplot     : 0.8.2
geoviews   : None
stonesoup  : None


In [57]:
# Read the CSV file and store it in a Pandas DataFrame
df = pd.read_csv("../Data/OpenSky/states_2020-05-25-00.csv", delimiter=',')
df

,time,icao24,lat,lon,velocity,heading,vertrate,callsign,onground,alert,spi,squawk,baroaltitude,geoaltitude,lastposupdate,lastcontact
0,1590364810,7c01c2,-34.505905,149.904243,130.920692,27.370563,0.00000,AM217,False,False,False,4067.0,5791.20,5897.88,1.590365e+09,1.590365e+09
1,1590364810,a1311e,35.968231,-115.362511,163.201227,203.008870,8.45312,SKW3355,False,False,False,6125.0,3398.52,3505.20,1.590365e+09,1.590365e+09
2,1590364810,7c6b18,-34.796401,149.516659,233.718992,55.855214,-12.35456,JST506,False,False,False,4334.0,10401.30,10546.08,1.590365e+09,1.590365e+09
3,1590364810,a600e4,29.091797,-97.094314,224.388419,295.096250,0.00000,SWA599,False,False,False,1417.0,12192.00,12679.68,1.590365e+09,1.590365e+09
4,1590364810,ac9ee1,48.748123,-122.360018,49.869237,227.090159,1.95072,N912J,False,False,False,1200.0,853.44,899.16,1.590365e+09,1.590365e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
723084,1590368390,a98c7a,43.652023,76.938853,239.855067,91.843648,0.00000,SOO098,False,False,False,6413.0,10058.40,10431.78,1.590368e+09,1.590368e+09
723085,1590368390,06a0e0,35.268394,26.474012,213.739565,274.832355,0.00000,QTR3261,False,False,False,5601.0,12192.00,12595.86,1.590368e+09,1.590368e+09
723086,1590368390,780750,22.808578,114.528725,164.668695,257.917745,-7.47776,CSZ9924,False,False,False,6300.0,3497.58,3741.42,1.590368e+09,1.590368e+09
723087,1590368390,a32f6e,33.432724,-104.651257,233.236111,128.462442,0.32512,AAY75,False,False,False,6450.0,11879.58,12245.34,1.590368e+09,1.590368e+09


In [58]:
df = df[df[['icao24', 'lat', 'lon','velocity','vertrate']].notnull().all(1)]

In [61]:
s= gpd.GeoSeries(gpd.points_from_xy(df.lat, df.lon))

In [62]:
s= s[~(s.is_empty | s.isna())]

In [63]:
s

0         POINT (-34.506 149.904)
1         POINT (35.968 -115.363)
2         POINT (-34.796 149.517)
3          POINT (29.092 -97.094)
4         POINT (48.748 -122.360)
                   ...           
576741      POINT (43.652 76.939)
576742      POINT (35.268 26.474)
576743     POINT (22.809 114.529)
576744    POINT (33.433 -104.651)
576745     POINT (41.807 -87.534)
Length: 576746, dtype: geometry

In [64]:
df

,time,icao24,lat,lon,velocity,heading,vertrate,callsign,onground,alert,spi,squawk,baroaltitude,geoaltitude,lastposupdate,lastcontact
0,1590364810,7c01c2,-34.505905,149.904243,130.920692,27.370563,0.00000,AM217,False,False,False,4067.0,5791.20,5897.88,1.590365e+09,1.590365e+09
1,1590364810,a1311e,35.968231,-115.362511,163.201227,203.008870,8.45312,SKW3355,False,False,False,6125.0,3398.52,3505.20,1.590365e+09,1.590365e+09
2,1590364810,7c6b18,-34.796401,149.516659,233.718992,55.855214,-12.35456,JST506,False,False,False,4334.0,10401.30,10546.08,1.590365e+09,1.590365e+09
3,1590364810,a600e4,29.091797,-97.094314,224.388419,295.096250,0.00000,SWA599,False,False,False,1417.0,12192.00,12679.68,1.590365e+09,1.590365e+09
4,1590364810,ac9ee1,48.748123,-122.360018,49.869237,227.090159,1.95072,N912J,False,False,False,1200.0,853.44,899.16,1.590365e+09,1.590365e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
723084,1590368390,a98c7a,43.652023,76.938853,239.855067,91.843648,0.00000,SOO098,False,False,False,6413.0,10058.40,10431.78,1.590368e+09,1.590368e+09
723085,1590368390,06a0e0,35.268394,26.474012,213.739565,274.832355,0.00000,QTR3261,False,False,False,5601.0,12192.00,12595.86,1.590368e+09,1.590368e+09
723086,1590368390,780750,22.808578,114.528725,164.668695,257.917745,-7.47776,CSZ9924,False,False,False,6300.0,3497.58,3741.42,1.590368e+09,1.590368e+09
723087,1590368390,a32f6e,33.432724,-104.651257,233.236111,128.462442,0.32512,AAY75,False,False,False,6450.0,11879.58,12245.34,1.590368e+09,1.590368e+09


In [65]:

df.time = pd.to_datetime(df['time'], unit='s')


In [66]:

df

,time,icao24,lat,lon,velocity,heading,vertrate,callsign,onground,alert,spi,squawk,baroaltitude,geoaltitude,lastposupdate,lastcontact
0,2020-05-25 00:00:10,7c01c2,-34.505905,149.904243,130.920692,27.370563,0.00000,AM217,False,False,False,4067.0,5791.20,5897.88,1.590365e+09,1.590365e+09
1,2020-05-25 00:00:10,a1311e,35.968231,-115.362511,163.201227,203.008870,8.45312,SKW3355,False,False,False,6125.0,3398.52,3505.20,1.590365e+09,1.590365e+09
2,2020-05-25 00:00:10,7c6b18,-34.796401,149.516659,233.718992,55.855214,-12.35456,JST506,False,False,False,4334.0,10401.30,10546.08,1.590365e+09,1.590365e+09
3,2020-05-25 00:00:10,a600e4,29.091797,-97.094314,224.388419,295.096250,0.00000,SWA599,False,False,False,1417.0,12192.00,12679.68,1.590365e+09,1.590365e+09
4,2020-05-25 00:00:10,ac9ee1,48.748123,-122.360018,49.869237,227.090159,1.95072,N912J,False,False,False,1200.0,853.44,899.16,1.590365e+09,1.590365e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
723084,2020-05-25 00:59:50,a98c7a,43.652023,76.938853,239.855067,91.843648,0.00000,SOO098,False,False,False,6413.0,10058.40,10431.78,1.590368e+09,1.590368e+09
723085,2020-05-25 00:59:50,06a0e0,35.268394,26.474012,213.739565,274.832355,0.00000,QTR3261,False,False,False,5601.0,12192.00,12595.86,1.590368e+09,1.590368e+09
723086,2020-05-25 00:59:50,780750,22.808578,114.528725,164.668695,257.917745,-7.47776,CSZ9924,False,False,False,6300.0,3497.58,3741.42,1.590368e+09,1.590368e+09
723087,2020-05-25 00:59:50,a32f6e,33.432724,-104.651257,233.236111,128.462442,0.32512,AAY75,False,False,False,6450.0,11879.58,12245.34,1.590368e+09,1.590368e+09


In [67]:
filtered_df = df[df[['icao24', 'lat', 'lon','velocity','vertrate']].notnull().all(1)]

In [68]:
gdf = gpd.GeoDataFrame(df, geometry=s)


In [69]:
gdf

,time,icao24,lat,lon,velocity,heading,vertrate,callsign,onground,alert,spi,squawk,baroaltitude,geoaltitude,lastposupdate,lastcontact,geometry
0,2020-05-25 00:00:10,7c01c2,-34.505905,149.904243,130.920692,27.370563,0.00000,AM217,False,False,False,4067.0,5791.20,5897.88,1.590365e+09,1.590365e+09,POINT (-34.506 149.904)
1,2020-05-25 00:00:10,a1311e,35.968231,-115.362511,163.201227,203.008870,8.45312,SKW3355,False,False,False,6125.0,3398.52,3505.20,1.590365e+09,1.590365e+09,POINT (35.968 -115.363)
2,2020-05-25 00:00:10,7c6b18,-34.796401,149.516659,233.718992,55.855214,-12.35456,JST506,False,False,False,4334.0,10401.30,10546.08,1.590365e+09,1.590365e+09,POINT (-34.796 149.517)
3,2020-05-25 00:00:10,a600e4,29.091797,-97.094314,224.388419,295.096250,0.00000,SWA599,False,False,False,1417.0,12192.00,12679.68,1.590365e+09,1.590365e+09,POINT (29.092 -97.094)
4,2020-05-25 00:00:10,ac9ee1,48.748123,-122.360018,49.869237,227.090159,1.95072,N912J,False,False,False,1200.0,853.44,899.16,1.590365e+09,1.590365e+09,POINT (48.748 -122.360)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
723084,2020-05-25 00:59:50,a98c7a,43.652023,76.938853,239.855067,91.843648,0.00000,SOO098,False,False,False,6413.0,10058.40,10431.78,1.590368e+09,1.590368e+09,None
723085,2020-05-25 00:59:50,06a0e0,35.268394,26.474012,213.739565,274.832355,0.00000,QTR3261,False,False,False,5601.0,12192.00,12595.86,1.590368e+09,1.590368e+09,None
723086,2020-05-25 00:59:50,780750,22.808578,114.528725,164.668695,257.917745,-7.47776,CSZ9924,False,False,False,6300.0,3497.58,3741.42,1.590368e+09,1.590368e+09,None
723087,2020-05-25 00:59:50,a32f6e,33.432724,-104.651257,233.236111,128.462442,0.32512,AAY75,False,False,False,6450.0,11879.58,12245.34,1.590368e+09,1.590368e+09,None


In [89]:
traj_collection = mpd.TrajectoryCollection(gdf, 'icao24', t='time')
gap = timedelta(minutes=5)


In [95]:
print(traj_collection)

TrajectoryCollection with 2980 trajectories


In [99]:
s= gpd.GeoDataFrame(gpd.points_from_xy(df.lat, df.lon))

In [107]:
split = mpd.ObservationGapSplitter(traj_collection).split(gap=timedelta(minutes=15))


AttributeError: 'NoneType' object has no attribute 'y'

In [93]:

split.add_speed(overwrite=True)

cleaned = mpd.OutlierCleaner(split).clean({'speed': 1})


NameError: name 'split' is not defined

In [ ]:
smoothed = mpd.KalmanSmootherCV(cleaned).smooth(process_noise_std=0.1, measurement_noise_std=10)

In [ ]:
print(smoothed)

In [ ]:
out = smoothed.to_point_gdf()


In [ ]:
output = out.to_csv('Output/aisdk-2022-11-18PointsMP.csv')
